In [18]:
import pandas as pd
from sklearn import preprocessing

In [19]:
# Read the raw data
data = pd.read_csv("../oslo_citybike_dataset.csv")
data.head(10)

,station_id,station_name,station_address,station_lat,station_lon,station_capacity,is_installed,is_renting,num_bikes_available,num_docks_available,is_returning,last_reported
0,2332,Legevakten,Storgata 38,59.916296,10.757377,12,1,1,0,12,1,1642601914
1,2330,Stjerneplassen,Storgata 10a,59.913233,10.749959,15,1,1,0,15,1,1642601914
2,2329,Via Vika,Dronning Mauds gate 10,59.912713,10.727171,15,1,1,2,13,1,1642601914
3,2328,The Hub,Biskop Gunnerus Gate 3,59.912522,10.750910,21,1,1,4,17,1,1642601914
4,2315,Rostockgata,Rostockgata 5,59.906920,10.760312,18,1,1,4,14,1,1642601914
5,2309,Ulven Torg,Ulvenveien 89,59.925035,10.812022,30,1,1,0,30,1,1642601914
6,2308,Gunerius,Storgata 33,59.914599,10.753399,12,1,1,3,9,1,1642601914
7,2307,Domus Athletica,Trimveien 4,59.946219,10.724626,30,1,1,0,30,1,1642601914
8,2306,Ã˜kern Portal,LÃ¸renfaret 1,59.930904,10.801940,18,1,1,1,16,1,1642601914
9,2305,Hesselbergs Gate,Toftes Gate 24,59.927748,10.761266,15,1,1,0,15,1,1642601914


In [20]:
# bike_availability_ratio
data["bike_availability_ratio"] = (data["num_bikes_available"] / data["station_capacity"]) * 100

# Datetime conversions
data["record_timestamp_converted"] = pd.to_datetime(data["last_reported"], unit='s', origin='unix')
data["record_timestamp_converted"] = data["record_timestamp_converted"].dt.strftime("%Y-%m-%d %H:%M:%S")
data['record_timestamp_year'] = pd.DatetimeIndex(data["record_timestamp_converted"]).year
data['record_timestamp_month'] = pd.DatetimeIndex(data["record_timestamp_converted"]).month
data['record_timestamp_day'] = pd.DatetimeIndex(data["record_timestamp_converted"]).day
data['record_timestamp_hour'] = pd.DatetimeIndex(data["record_timestamp_converted"]).hour
data['record_timestamp_minute'] = pd.DatetimeIndex(data["record_timestamp_converted"]).minute
data['record_timestamp_second'] = pd.DatetimeIndex(data["record_timestamp_converted"]).second

# Geolocation normalization
data["station_lat"] = data["station_lat"].apply(lambda x: x / 100)
data["station_lon"] = data["station_lon"].apply(lambda x: x / 100)
data.head()

,station_id,station_name,station_address,station_lat,station_lon,station_capacity,is_installed,is_renting,num_bikes_available,num_docks_available,is_returning,last_reported,bike_availability_ratio,record_timestamp_converted,record_timestamp_year,record_timestamp_month,record_timestamp_day,record_timestamp_hour,record_timestamp_minute,record_timestamp_second
0,2332,Legevakten,Storgata 38,0.599163,0.107574,12,1,1,0,12,1,1642601914,0.000000,2022-01-19 14:18:34,2022,1,19,14,18,34
1,2330,Stjerneplassen,Storgata 10a,0.599132,0.107500,15,1,1,0,15,1,1642601914,0.000000,2022-01-19 14:18:34,2022,1,19,14,18,34
2,2329,Via Vika,Dronning Mauds gate 10,0.599127,0.107272,15,1,1,2,13,1,1642601914,13.333333,2022-01-19 14:18:34,2022,1,19,14,18,34
3,2328,The Hub,Biskop Gunnerus Gate 3,0.599125,0.107509,21,1,1,4,17,1,1642601914,19.047619,2022-01-19 14:18:34,2022,1,19,14,18,34
4,2315,Rostockgata,Rostockgata 5,0.599069,0.107603,18,1,1,4,14,1,1642601914,22.222222,2022-01-19 14:18:34,2022,1,19,14,18,34


In [21]:
# Drop redundant columns
df = data.drop(labels=["station_id", "station_name", "station_address", "station_capacity", "num_bikes_available", "num_docks_available",
                       "last_reported", "record_timestamp_converted"], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2223 entries, 0 to 2222
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   station_lat              2223 non-null   float64
 1   station_lon              2223 non-null   float64
 2   is_installed             2223 non-null   int64  
 3   is_renting               2223 non-null   int64  
 4   is_returning             2223 non-null   int64  
 5   bike_availability_ratio  2223 non-null   float64
 6   record_timestamp_year    2223 non-null   int64  
 7   record_timestamp_month   2223 non-null   int64  
 8   record_timestamp_day     2223 non-null   int64  
 9   record_timestamp_hour    2223 non-null   int64  
 10  record_timestamp_minute  2223 non-null   int64  
 11  record_timestamp_second  2223 non-null   int64  
dtypes: float64(3), int64(9)
memory usage: 208.5 KB


In [22]:
le = preprocessing.LabelEncoder()
df["is_installed"] = le.fit_transform(df["is_installed"])
df["is_renting"] = le.fit_transform(df["is_renting"])
df["is_returning"] = le.fit_transform(df["is_returning"])

In [23]:
df = pd.get_dummies(df, columns=["record_timestamp_year", "record_timestamp_month", "record_timestamp_day", "record_timestamp_hour", "record_timestamp_minute", "record_timestamp_second"])
df.head()

,station_lat,station_lon,is_installed,is_renting,is_returning,bike_availability_ratio,record_timestamp_year_2022,record_timestamp_month_1,record_timestamp_day_19,record_timestamp_hour_14,...,record_timestamp_minute_36,record_timestamp_second_24,record_timestamp_second_26,record_timestamp_second_30,record_timestamp_second_33,record_timestamp_second_34,record_timestamp_second_36,record_timestamp_second_43,record_timestamp_second_46,record_timestamp_second_59
0,0.599163,0.107574,0,0,0,0.000000,1,1,1,1,...,0,0,0,0,0,1,0,0,0,0
1,0.599132,0.107500,0,0,0,0.000000,1,1,1,1,...,0,0,0,0,0,1,0,0,0,0
2,0.599127,0.107272,0,0,0,13.333333,1,1,1,1,...,0,0,0,0,0,1,0,0,0,0
3,0.599125,0.107509,0,0,0,19.047619,1,1,1,1,...,0,0,0,0,0,1,0,0,0,0
4,0.599069,0.107603,0,0,0,22.222222,1,1,1,1,...,0,0,0,0,0,1,0,0,0,0


In [24]:
df.to_csv('../oslo_citybike_dataset_preprocessed.csv', index=False)